# Electric Vehicle Routing Problem 1 -- Zielfunktion und Datenstruktur

Website des Wettbewerbs: https://mavrovouniotis.github.io/EVRPcompetition2020/

Problembeschreibung: https://mavrovouniotis.github.io/EVRPcompetition2020/TR-EVRP-Competition.pdf


Wenn Sie den Code in colab ausführen, dann müssen Sie **nichts** weiter tun: Data und Helfercode werden automatisch herunter geladen. Wenn Sie auf Ihrem eigenen Laptop arbeiten, laden Sie Daten und Helfercode unter 
* Daten: https://mavrovouniotis.github.io/EVRPcompetition2020/evrp-benchmark-set.zip
* Helfercode: https://raw.githubusercontent.com/henningbruhn/mobp/main/vrp_aufgabe/evrp.py

herunter, entpacken Sie die Daten und speichern Sie die Daten in einem Verzeichnis namens <code>data</code>. Das Verzeichnis <code>data</code> sowie <code>evrp.py</code> müssen im gleichen Verzeichnis wie dieses Notebook liegen. Kommentieren Sie schließlich die markierten Zeilen in der nächsten Zelle aus.

In [ ]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import time

path="data"

#### Wenn Sie den Code auf dem eigenen Laptop ausführen, kommentieren Sie die folgenden Zeilen aus ####
# download visualisation and helper code
!wget -q https://raw.githubusercontent.com/henningbruhn/mobp/main/vrp_aufgabe/evrp.py

# download and unpack instance data
!wget -q https://mavrovouniotis.github.io/EVRPcompetition2020/evrp-benchmark-set.zip
!unzip -qo evrp-benchmark-set.zip -d data
#### Ende auskommentieren ####

import evrp

Wir lesen die Instanzen ein und lassen uns die Namen der Instanzen ausgeben.

In [ ]:
instances=evrp.read_in_all_instances(path)
for key in instances.keys():
    print("{:12} -> {:4} Kunden".format(key,instances[key].num_customers))

Wir picken uns eine der kleinen Instanzen heraus und lassen uns die Instanz anzeigen.

In [ ]:
inst=instances['E-n33-k4']
evrp.show(inst)

Das <code>inst</code>-Objekt speichert alle Daten der Instanz. Insbesondere verfügt es über Listen <code>customers</code> und <code>stations</code>. Die Kunden und Ladestationen sind einfach durchnummeriert. Das Depot trägt die Nummer 1 und gilt ebenfalls als Ladestation.

In [ ]:
print("ersten 10 Kunden: {}".format(inst.customers[:10]))
print("Ladestationen   : {}".format(inst.stations))

Wir können uns die 2d-Koordinaten jedes Kunden und jeder Station wie folgt ausgeben lassen (hier Station 37):

In [ ]:
inst.nodes[37]

Wir lassen uns alle Attribute des Instanzobjekts anzeigen:

In [ ]:
print(inst)

## Exploration der Daten

Bei jeder datengetriebenen Fragestellung ist es unerlässlich, sich ein Überblick über die Daten zu verschaffen. Oben haben wir uns schon die Anzahl der Kunden pro Instanz auflisten lassen. Das ist sinnvoll, da wir so eine erste Idee der Schwere der Aufgabe erhalten. Welche weitere Kennzahlen sollten wir erheben? Hier sind zwei Vorschläge:
* die durchschnittliche Anzahl an Kunden pro Tour (dh, pro Fahrzeug)
* die Mindestzahl an benötigten Fahrzeuge und die tatsächlich vorhandene Anzahl

Wie können wir die Mindestzahl abschätzen? Wir rechnen die Gesamtsumme des Bedarfs der Kunden zusammen und teilen durch die Kapazität des Fahrzeuge.

Noch ein Tipp: Wenn Sie über die Instanzen iterieren wollen, können Sie dies so machen:

```
for instance in instances.values():
  ...do something
```

Erheben Sie sinnvolle Kennzahlen!

In [ ]:
### Fügen Sie Ihren Code hier ein ###

## Touren

Wie stellen wir eine Tour, also eine Lösung des Problems dar? Als Liste von Listen. Jedes Fahrzeug wird durch eine Liste repräsentiert, deren Einträge die angefahrenen Kunden bzw. Ladestationen widergeben. Gucken wir uns das mal an. Dazu erzeugen wir eine Zufallslösung mittels der Funktion <code>evrp.rnd_tour(instance, num_stations_insert=123)</code>. Dabei bestimmt der Parameter <code>num_stations_insert</code> wie viele Ladestation zufällig angefahren werden sollen.

In [ ]:
random_tour=evrp.rnd_tour(inst,num_stations_insert=3)
random_tour

Mit <code>evrp.tour_length(random_tour,inst)</code> können wir die Gesamtlänge der Tour bestimmen. Die Funktion <code>evrp.show</code> kann auch die Tour visualisieren.

In [ ]:
length=evrp.tour_length(random_tour,inst)
print("Gesamtlänge der Touren: {:.1f}".format(length))
evrp.show(inst,tour=random_tour)

Wir sehen: Die Zufallslösung ist nicht zulässig. Vielleicht war das einfach nur Pech? Erzeugen Sie 1000 Zufallstouren und bestimmen Sie wie viele davon zulässig sind (also die Laderaum- und Batteriekapazität nicht überschreiten). Nutzen Sie dazu die Funktion <code>evrp.validate(tour,instance)</code>, die <code>True</code> zurück liefert, wenn <code>tour</code> zulässig ist. 

Ist die Erzeugung von Zufallslösungen vielversprechend um eine gute Lösungen zu produzieren?

In [ ]:
### Fügen Sie Ihren Code hier ein ###

## Zielfunktion

Unten sind drei Touren (Lösungen) bereit gestellt. Zwei davon sind nicht zulässig. Wenn wir die Qualität der Touren beurteilen wollen, also eine Zielfunktion aufstellen, so wird die zulässige Tour die höchste Qualität haben (kleinsten Zielfunktionswert). Wie aber unterscheiden wir die Qualität der beiden unzulässigen Touren? Stellen Sie eine Zielfunktion auf, die die drei Touren in eine sinnvolle Reihenfolge bringt. 

Nützlich dazu sind die folgenden Funktionen:
* <code>evrp.tour_length(tour,inst)</code>: Die Gesamtlänge der Tour.
* <code>evrp.compute_loads(tour,inst)</code>: Eine Liste mit einem Eintrag pro Fahrzeug; jeder Eintrag ist der Gesamtbedarf der Kunden, die dem Fahrzeug zugeordnet sind.
* <code>evrp.compute_charge_lvls(tour,inst)</code>: Eine Liste von Listen, gibt den Batterieladungsstand jedes Fahrzeugs bei jedem Kunden an. Negative Ladungsstände zeigen an, dass die Reichweite überschritten ist.

In [ ]:
tour1=evrp.sample1()
tour2=evrp.sample2()
tour3=evrp.sample3()

In [ ]:
### Fügen Sie Ihren Code hier ein ###